In [1]:
import bs4
import lxml.html
from selenium.common.exceptions import NoSuchElementException
from lxml import etree
from selenium import webdriver
import selenium
import requests
import csv

In [21]:
driver = webdriver.Chrome(executable_path=r'C:\Users\Akshaya\Downloads\chromedriver.exe')

In [3]:
vanames = []

In [4]:
links = ["https://en.wikipedia.org/wiki/Category:Japanese_voice_actresses","https://en.wikipedia.org/w/index.php?title=Category:Japanese_voice_actresses&pagefrom=Iijima%2C+Mari%0AMari+Iijima#mw-pages","https://en.wikipedia.org/w/index.php?title=Category:Japanese_voice_actresses&pagefrom=Machico%0AMachico+%28singer%29#mw-pages","https://en.wikipedia.org/w/index.php?title=Category:Japanese_voice_actresses&pagefrom=Okasaki%2C+Miho%0AMiho+Okasaki#mw-pages","https://en.wikipedia.org/w/index.php?title=Category:Japanese_voice_actresses&pagefrom=Takashima%2C+Reiko%0AReiko+Takashima#mw-pages","https://en.wikipedia.org/wiki/Category:Japanese_male_voice_actors","https://en.wikipedia.org/w/index.php?title=Category:Japanese_male_voice_actors&pagefrom=Kanao%2C+Tetsuo%0ATetsuo+Kanao#mw-pages","https://en.wikipedia.org/w/index.php?title=Category:Japanese_male_voice_actors&pagefrom=Nishimoto%2C+Hiroyuki%0AHiroyuki+Nishimoto#mw-pages","https://en.wikipedia.org/w/index.php?title=Category:Japanese_male_voice_actors&pagefrom=Tatekabe%2C+Kazuya%0AKazuya+Tatekabe#mw-pages"]

for link in links:
    driver.get(link)
    for j in range(1,27):
        i=1
        while(True):
            try:
                name = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[4]/div[2]/div[2]/div/div/div['+str(j)+']/ul/li['+str(i)+']/a')
                vanames.append(name.text)
                i+=1
            except NoSuchElementException:
                break

In [5]:
va_names=[]

In [6]:
for name in vanames:
    if "ō" in name:
        va_names.append(name.replace("ō","ou"))
        va_names.append(name.replace("ō","o"))
    else:
        va_names.append(name)

In [7]:
def nameconv(name):
    name = name.split(" ")
    if len(name)>1:
        name[0] = name[0][:-1]
        name = name[1] + " " + name[0]
        return(name)

In [8]:
names_va = []
names_links = {}
for i in range(0,8):
    link = "https://myanimelist.net/people.php?limit=" + str(50*i)
    driver.get(link)
    i=2
    while(True):
        try:
            name = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div[2]/table/tbody/tr['+str(i)+']/td[2]/div/a')
            if nameconv(name.text) in va_names:
                names_va.append(nameconv(name.text))
                names_links[nameconv(name.text)] = name.get_attribute("href")
            i+=1
        except NoSuchElementException:
            break

In [9]:
ani_names = []
link = "https://myanimelist.net/animelist/animefankitty"
driver.get(link)
i=2
while(True):
    try:
        name = driver.find_element_by_xpath('/html/body/div[3]/div[4]/div/table/tbody['+str(i)+']/tr[1]/td[4]/a')
        ani_names.append(name.text)
        i+=1
    except NoSuchElementException:
        break

In [31]:
with open('ani_list.csv', mode='w', encoding="utf-8") as anilist:
    anilist = csv.writer(anilist, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for x in ani_names:
        anilist.writerow([x])

In [18]:
with open('va_list.csv', mode='w', encoding="utf-8") as valist:
    valist = csv.writer(valist, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for x in names_va:
        valist.writerow([x])

In [19]:
va_list = {}

In [37]:
for vname in names_va[:]:
    link = names_links[vname]
    driver.get(link)
    va_list[vname] = {}
    i=1
    while(True):
        try:
            aniname = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div[2]/table/tbody/tr/td[2]/table[1]/tbody/tr['+str(i)+']/td[2]/a')
            charname = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div[2]/table/tbody/tr/td[2]/table[1]/tbody/tr['+str(i)+']/td[3]/a')
            if aniname.text in ani_names:
                va_list[vname][aniname.text] = charname.text
            i+=1
        except NoSuchElementException:
            break

In [38]:
with open('aniva_list.csv', mode='w', encoding="utf-8") as anivalist:
    anivalist = csv.writer(anivalist, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for key, val in va_list.items():
        anivalist.writerow([key, val])

In [51]:
with open('aniva_list.txt', mode='w', encoding="utf-8") as anivalist:
    for name in names_va:
        anivalist.write(name)
        anivalist.write("\n")
        anivalist.write("\n")
        anivalist.write(str(va_list[name]))
        anivalist.write("\n")
        anivalist.write("\n")
        anivalist.write("\n")